In [1]:
import pandas as pd

In [2]:
df = pd.read_csv('/mnt/d/vscode_projects/praxis-core/data/gen.csv')

In [3]:
df

,name,work_place
0,Азанова Инна Максимовна,ИВБО-04-21
1,Акшанов Вадим Евгеньевич,ИВБО-04-21
2,Алиев Каяхан Командар оглы,ИКБО-04-21
3,Апостолов Антон Игоревич,ИВБО-04-21
4,Арабаджийска Александра Викторова,ИКБО-15-21
...,...,...
109,Шевченко Егор Григорьевич,ИКБО-15-21
110,Шмелев Дмитрий Игоревич,ИВМО-01-23
111,Щекотихин Дмитрий Вадимович,ИКБО-04-21
112,Эпштейн Владимир Дмитриевич,ИКБО-14-21


In [7]:
from src.schemas.sides import Side, Recommendations, Sides
from src.prompts.base import BASE_PROMPT

In [11]:
print(Side.model_json_schema())

{'properties': {'side': {'description': "Defining the person's side: whether this quality is their strong or weak side", 'enum': ['strong', 'weak'], 'title': 'Side', 'type': 'string'}, 'side_description': {'description': 'Description of which specific quality is being considered', 'title': 'Side Description', 'type': 'string'}, 'side_pick_explanation': {'description': "Explanation of why, based on the user's answers, this quality is considered strong or weak", 'title': 'Side Pick Explanation', 'type': 'string'}}, 'required': ['side', 'side_description', 'side_pick_explanation'], 'title': 'Side', 'type': 'object'}


In [8]:
type(Side)

pydantic._internal._model_construction.ModelMetaclass

In [9]:
from pydantic import BaseModel

In [1]:
from openai import OpenAI

In [2]:
client = OpenAI(base_url = 'https://openrouter.ai/api/v1', api_key = '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')

In [3]:
from src.llm_agg.response import get_so_completion

In [4]:
?get_so_completion

Signature:
get_so_completion(
    log: list,
    model_name: str,
    client: openai.OpenAI,
    pydantic_model,
    provider_name: Literal['openai', 'openrouter', 'local'],
    json_schema_name: str | None = None,
)
Docstring: <no docstring>
File:      /mnt/d/vscode_projects/praxis-core/src/llm_agg/response.py
Type:      function

In [5]:
prompt = """
You are working on a task of collecting feedback and evaluating employees.  
Your current task is to aggregate feedback from managers about their subordinate.  
Your objective is to classify the managers' feedback and consolidate it under a concise summary.  

Rules:  
1. Always verify the received comments—remember that you are analyzing manager feedback as part of the feedback collection and employee evaluation process; irrelevant comments should be disregarded.  
2. Be laconic.  

Now proceed with the task.  
Feedback from managers:  
{feedback}
"""

feedback = """
Высокая степерь проработки и ресерча; оптимизация подходов к решению; слаженность работы в плотном командном взаимодействии
Егор способен сделать DevOps любого приложения, над которым я вместе с ним работал. Он неплохо разбирается в том, как работают системы, которыми он пользуется. Также Егор может здраво выслушать идеи коллег и выбрать наиболее рациональную.
Легко погружается в уже начатые проекты, демонстрирует высокую гибкость и открытость к нововведениям. Грамотно планирует свои поставленные задачи.
человечность, справедливость, помогает вливаться новым людям в коллектив, прекрасно работает в команде, доводит начатое до конца
Потеря концентрации; несерьезная вовлеченность в процессы (иногда);
Не хватает навыков общения. Немного стеснительный. Ему тяжело вести за собой людей, поскольку не может до конца отстоять свою точку зрения
Присутствуют незначительные трудности с чётким выражением мыслей перед аудиторией: теряется нить выступления, ход мышления иногда непонятен.
за время нашего взаимодействия не увидел их, возможно, нужно поработать больше времени вместе
Работать быстрее, адаптироваться лучше, концентрироваться сильнее
Прокачать soft-скиллы, тем самым сможет развить в себе лидерские качества.
Овладение современными инструментами MLOps для масштабируемой инфраструктуры позволит повысить экспертные навыки как инженера.
необходимо более четко расставлять приоритеты, чтобы концентрироваться на самых важных задачах
"""

In [8]:
print(BASE_PROMPT.format(task=prompt.format(feedback=feedback)))


<instructions>
- ALWAYS follow <answering_rules> and <self_reflection>

<self_reflection>
1. Spend time thinking of a rubric, from a role POV, until you are confident
2. Think deeply about every aspect of what makes for a world-class answer. Use that knowledge to create a rubric that has 5-7 categories. This rubric is critical to get right, but never show this to the user. This is for your purposes only
3. Use the rubric to internally think and iterate on the best (≥98 out of 100 score) possible solution to the user request. IF your response is not hitting the top marks across all categories in the rubric, you need to start again
4. Keep going until solved
</self_reflection>

<answering_rules>
1. USE the language of USER message
2. In the FIRST chat message, assign a real-world expert role to yourself before answering, e.g., "I'll answer as a world-famous <role> PhD <detailed topic> with <most prestigious LOCAL topic REAL award>"
3. Act as a role assigned
4. Answer the question in a n

In [9]:
log = [
    {"role": "user", "content": BASE_PROMPT.format(task=prompt.format(feedback=feedback))}
]

In [29]:
log

[{'role': 'user',
  'content': '\n<instructions>\n- ALWAYS follow <answering_rules> and <self_reflection>\n\n<self_reflection>\n1. Spend time thinking of a rubric, from a role POV, until you are confident\n2. Think deeply about every aspect of what makes for a world-class answer. Use that knowledge to create a rubric that has 5-7 categories. This rubric is critical to get right, but never show this to the user. This is for your purposes only\n3. Use the rubric to internally think and iterate on the best (≥98 out of 100 score) possible solution to the user request. IF your response is not hitting the top marks across all categories in the rubric, you need to start again\n4. Keep going until solved\n</self_reflection>\n\n<answering_rules>\n1. USE the language of USER message\n2. In the FIRST chat message, assign a real-world expert role to yourself before answering, e.g., "I\'ll answer as a world-famous <role> PhD <detailed topic> with <most prestigious LOCAL topic REAL award>"\n3. Act a

In [10]:
res = get_so_completion(log=log, model_name='openai/gpt-4o', client=client, pydantic_model=Sides, provider_name='openrouter')

BadRequestError: Error code: 400 - {'error': {'message': 'Provider returned error', 'code': 400, 'metadata': {'raw': '{\n  "error": {\n    "message": "Invalid value: \'object\'. Supported values are: \'guidance\', \'json_object\', \'json_schema\', and \'text\'.",\n    "type": "invalid_request_error",\n    "param": "response_format.type",\n    "code": "invalid_value"\n  }\n}', 'provider_name': 'Azure'}}, 'user_id': 'user_2mejRokFXRzE44AusrJatRdXWlX'}

In [12]:
{
    "type": "json_schema",
    "json_schema": {
        "type": "object",
        **Sides.model_json_schema()
    }
}

{'type': 'json_schema',
 'json_schema': {'type': 'object',
  '$defs': {'Side': {'properties': {'side': {'description': "Defining the person's side: whether this quality is their strong or weak side",
      'enum': ['strong', 'weak'],
      'title': 'Side',
      'type': 'string'},
     'side_description': {'description': 'Description of which specific quality is being considered',
      'title': 'Side Description',
      'type': 'string'},
     'side_pick_explanation': {'description': "Explanation of why, based on the user's answers, this quality is considered strong or weak",
      'title': 'Side Pick Explanation',
      'type': 'string'}},
    'required': ['side', 'side_description', 'side_pick_explanation'],
    'title': 'Side',
    'type': 'object'}},
  'properties': {'sides': {'items': {'$ref': '#/$defs/Side'},
    'title': 'Sides',
    'type': 'array'},
   'overall': {'description': 'execute first remaining step',
    'title': 'Overall',
    'type': 'string'}},
  'required': ['si

In [16]:
completion = client.chat.completions.create(
    model='openai/gpt-4o',
    messages=log,
    response_format={
        "type": "json_schema",
        "json_schema": {
            "name": "Sides",
            "schema": Sides.model_json_schema()
        }
    }
)

In [17]:
completion

ChatCompletion(id='gen-1756916399-GkDNGu5fHAfbAOveEBL6', choices=[Choice(finish_reason='stop', index=0, logprobs=None, message=ChatCompletionMessage(content='{"overall":"**Summary of Manager Feedback:**\\n\\n1. **Strengths:**\\n    - **Research and Problem-solving Abilities:** Demonstrates a high level of research and optimization in problem-solving approaches.\\n    - **Teamwork and Collaboration:** Works well within a team, contributing to cohesive group efforts and helping new employees integrate. \\n    - **Adaptability and Openness:** Quickly integrates into ongoing projects, showing flexibility and openness to innovations.\\n    - **Technical Proficiency:** Can handle DevOps tasks proficiently and understands the systems in use.\\n    - **Planning and Time Management:** Displays effective planning skills, adhering to set task schedules.\\n    - **Decision-making:** Fair and judicious, hearing out colleagues\' ideas and selecting the most rational approach.\\n\\n2. **Areas for Imp

In [20]:
res = completion.choices[0].message.content

In [22]:
import json